# 🎯 SmartEye 학습지 분석 시스템 - Gradio UI 버전

## 📋 개요
DocLayout-YOLO와 OpenAI Vision API를 활용한 학습지 분석 시스템의 Gradio 웹 UI 버전입니다.

## 🚀 주요 기능
1. **LAM (Layout Analysis Module)**: DocLayout-YOLO를 사용한 레이아웃 분석
2. **TSPM (Text/Image Processing Module)**: OCR 및 AI API를 통한 콘텐츠 분석
3. **CIM (Content Integration Module)**: 결과 통합 및 문서화
4. **Gradio Web UI**: 직관적인 웹 인터페이스

## ⚙️ 실행 환경
- Google Colab (GPU 권장)
- Python 3.7+
- CUDA (선택사항)

## 📝 사용법
1. 모든 셀을 순서대로 실행
2. 마지막 셀 실행 후 생성되는 Gradio 링크 클릭
3. 웹 UI에서 단계별로 분석 진행

In [ ]:
# ============================================
# 📦 환경 설정 및 패키지 설치
# ============================================

print("🔧 SmartEye 환경 설정을 시작합니다...")

# 기본 패키지 설치
!pip install -q numpy opencv-python matplotlib pillow tqdm
!pip install -q gradio huggingface_hub loguru rich
!pip install -q transformers openai requests

# Tesseract OCR 설치
!apt-get update -qq
!apt-get install -qq -y tesseract-ocr tesseract-ocr-kor
!pip install -q pytesseract

# DocLayout-YOLO 설치
import os
if not os.path.exists('DocLayout-YOLO'):
    !git clone https://github.com/opendatalab/DocLayout-YOLO.git

%cd DocLayout-YOLO
!pip install -q -e .

print("✅ 모든 패키지 설치가 완료되었습니다!")

🔧 SmartEye 환경 설정을 시작합니다...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.0 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package tesseract-ocr-kor.
(Reading database ... 126111 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-kor_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-kor (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-kor (1:4.00~git30-7274cfa-1.1) ...
Cloning into 'DocLayout-YOLO'...
remote: Enumerating objects: 408, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 408 (delta 58), reused 41 (delta 38), pack-reused 305 (from 1)
Receiving objects: 100% (408/408), 11.78 MiB | 16.77 MiB/s, done.
Resolving deltas: 100% (125/125), done.
/content/DocLayout-YO

In [ ]:
# ============================================
# 📥 라이브러리 임포트
# ============================================

import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
import torch
import time
import base64
import io
import requests
import colorsys
from PIL import Image, ImageDraw, ImageFont
from tqdm.notebook import tqdm
import pytesseract
import gradio as gr
from loguru import logger
from huggingface_hub import hf_hub_download
from doclayout_yolo import YOLOv10

# 로그 설정
logger.remove()
logger.add(sys.stderr, level="INFO")

# 전역 변수 초기화
global_model = None
global_device = None
global_image_path = None
global_img = None
global_layout_info = None
global_ocr_results = []
global_api_results = []

logger.info("✅ 라이브러리 임포트 완료!")
logger.info(f"🖥️ 사용 가능한 디바이스: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

2025-06-04 04:58:04.780 | INFO     | __main__:<cell line: 0>:38 - ✅ 라이브러리 임포트 완료!
2025-06-04 04:58:04.825 | INFO     | __main__:<cell line: 0>:39 - 🖥️ 사용 가능한 디바이스: CUDA


In [ ]:
# ============================================
# 🛠️ SmartEye 핵심 함수들
# ============================================

def setup_environment():
    """환경 설정 함수"""
    try:
        device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        gpu_info = ""

        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
            gpu_info = f"\n🎮 GPU: {gpu_name}\n💾 GPU 메모리: {gpu_memory:.1f} GB"

        return f"✅ 환경 설정 완료!\n🖥️ 디바이스: {device}{gpu_info}\n📦 모든 라이브러리 로드됨\n🔧 Tesseract OCR 준비됨"
    except Exception as e:
        return f"❌ 환경 설정 실패: {e}"

def download_model_real(model_choice):
    """실제 모델 다운로드"""
    global global_model, global_device

    try:
        models = {
            "DocStructBench (학습지 최적화)": {
                "repo_id": "juliozhao/DocLayout-YOLO-DocStructBench",
                "filename": "doclayout_yolo_docstructbench_imgsz1024.pt"
            },
            "DocLayNet-Docsynth300K (일반문서)": {
                "repo_id": "juliozhao/DocLayout-YOLO-DocLayNet-Docsynth300K_pretrained",
                "filename": "doclayout_yolo_doclaynet_imgsz1120_docsynth_pretrain.pt"
            },
            "DocSynth300K-pretrain (커스텀)": {
                "repo_id": "juliozhao/DocLayout-YOLO-DocSynth300K-pretrain",
                "filename": "doclayout_yolo_docsynth300k_imgsz1600.pt"
            }
        }

        selected_model = models.get(model_choice, models["DocStructBench (학습지 최적화)"])

        logger.info(f"⬇️ 모델 다운로드 시작: {selected_model['filename']}")

        # 모델 다운로드
        model_path = hf_hub_download(
            repo_id=selected_model["repo_id"],
            filename=selected_model["filename"]
        )

        # 모델 로드
        global_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        global_model = YOLOv10(model_path, task='predict')
        global_model.to(global_device)

        logger.success(f"✅ 모델 로드 완료: {model_path}")

        return f"✅ 모델 다운로드 및 로드 완료!\n📁 경로: {model_path}\n🤖 모델: {model_choice}\n🖥️ 디바이스: {global_device}"

    except Exception as e:
        logger.error(f"❌ 모델 다운로드 실패: {e}")
        return f"❌ 모델 다운로드 실패: {e}"

def process_uploaded_image(image):
    """업로드된 이미지 처리"""
    global global_image_path, global_img

    try:
        if image is None:
            return None, "❌ 이미지를 업로드해주세요!"

        # PIL 이미지를 OpenCV 형식으로 변환
        img_array = np.array(image)
        global_img = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

        # 임시 파일로 저장 (모델 입력용)
        global_image_path = "/tmp/uploaded_image.jpg"
        cv2.imwrite(global_image_path, global_img)

        height, width = global_img.shape[:2]
        file_size = os.path.getsize(global_image_path) / 1024  # KB

        logger.info(f"✅ 이미지 처리 완료: {width}x{height}, {file_size:.1f}KB")

        return image, f"✅ 이미지 처리 완료!\n📐 크기: {width} x {height}\n💾 파일 크기: {file_size:.1f} KB\n📁 저장 경로: {global_image_path}"

    except Exception as e:
        logger.error(f"❌ 이미지 처리 실패: {e}")
        return None, f"❌ 이미지 처리 실패: {e}"

def visualize_layout_results(img, layout_info, alpha=0.3):
    """레이아웃 결과 시각화"""
    overlay = img.copy()

    # 클래스별 고유 색상 생성
    unique_classes = list(set(layout['class_name'] for layout in layout_info))
    class_colors = {}

    for i, cls_name in enumerate(unique_classes):
        h = i / max(1, len(unique_classes))
        s = 0.8
        v = 0.9
        r, g, b = colorsys.hsv_to_rgb(h, s, v)
        class_colors[cls_name] = (int(b * 255), int(g * 255), int(r * 255))

    # 클래스별 카운터
    class_counters = {cls: 0 for cls in unique_classes}

    for layout in layout_info:
        x1, y1, x2, y2 = layout['box']
        cls_name = layout['class_name']
        class_counters[cls_name] += 1
        class_id = class_counters[cls_name]

        color = class_colors[cls_name]

        # 바운딩 박스 내부 채우기
        cv2.rectangle(overlay, (x1, y1), (x2, y2), color, -1)

        # 바운딩 박스 테두리
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

        # 라벨 텍스트
        label = f"{cls_name} {class_id} ({layout['confidence']:.2f})"
        labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        y1_label = max(y1, labelSize[1] + 10)

        cv2.rectangle(img, (x1, y1_label - labelSize[1] - 10),
                     (x1 + labelSize[0], y1_label), color, -1)
        cv2.putText(img, label, (x1, y1_label - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # 오버레이 적용
    result = cv2.addWeighted(overlay, alpha, img, 1-alpha, 0)
    return result

def analyze_layout_real(confidence_threshold=0.25):
    """실제 레이아웃 분석"""
    global global_model, global_device, global_image_path, global_img, global_layout_info

    try:
        if global_model is None:
            return None, "❌ 먼저 모델을 다운로드해주세요!"

        if global_image_path is None:
            return None, "❌ 먼저 이미지를 업로드해주세요!"

        logger.info("🔍 레이아웃 분석 시작...")

        # 레이아웃 분석 실행
        results = global_model.predict(
            global_image_path,
            imgsz=1024,
            conf=confidence_threshold,
            iou=0.45,
            device=global_device
        )

        # 결과 파싱
        boxes = results[0].boxes.xyxy.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        confs = results[0].boxes.conf.cpu().numpy()
        class_names = global_model.names

        global_layout_info = []

        for i, (box, cls, conf) in enumerate(zip(boxes, classes, confs)):
            x1, y1, x2, y2 = map(int, box)
            cls_id = int(cls)
            cls_name = class_names.get(cls_id, f"unknown_{cls_id}")

            # 면적 필터링
            area = (x2 - x1) * (y2 - y1)
            if area < 100:  # 최소 면적
                continue

            global_layout_info.append({
                'id': i,
                'class_name': cls_name,
                'confidence': float(conf),
                'box': [x1, y1, x2, y2],
                'coordinates': [x1, y1, x2, y2],
                'area': area
            })

        # 결과 시각화
        result_img = visualize_layout_results(global_img.copy(), global_layout_info)
        result_pil = Image.fromarray(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))

        # 클래스별 통계
        class_counts = {}
        for layout in global_layout_info:
            cls = layout['class_name']
            class_counts[cls] = class_counts.get(cls, 0) + 1

        stats_text = f"✅ 레이아웃 분석 완료!\n📊 총 {len(global_layout_info)}개 영역 감지\n🎯 신뢰도 임계값: {confidence_threshold}\n\n📋 클래스별 개수:\n"

        for cls, count in sorted(class_counts.items()):
            stats_text += f"   • {cls}: {count}개\n"

        logger.success(f"✅ 레이아웃 분석 완료: {len(global_layout_info)}개 영역")

        return result_pil, stats_text

    except Exception as e:
        logger.error(f"❌ 레이아웃 분석 실패: {e}")
        return None, f"❌ 레이아웃 분석 실패: {e}"

def run_ocr_analysis(language="kor+eng"):
    """OCR 분석 실행"""
    global global_layout_info, global_img, global_ocr_results

    try:
        if global_layout_info is None:
            return "❌ 먼저 레이아웃 분석을 실행해주세요!"

        logger.info(f"📝 OCR 분석 시작: {language}")

        # OCR 대상 클래스
        target_classes = ['title', 'plain text', 'abandon text', 'table caption', 'table footnote', 'isolated formula', 'formula caption']
        global_ocr_results = []

        for layout in global_layout_info:
            if layout['class_name'].lower() not in target_classes:
                continue

            x1, y1, x2, y2 = layout['box']

            # 좌표 유효성 검사
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(global_img.shape[1], x2)
            y2 = min(global_img.shape[0], y2)

            cropped_img = global_img[y1:y2, x1:x2]

            if cropped_img.size == 0:
                continue

            # OCR 실행
            pil_img = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
            text = pytesseract.image_to_string(pil_img, lang=language).strip()

            if len(text) > 1:  # 노이즈 필터링
                global_ocr_results.append({
                    'id': layout['id'],
                    'class_name': layout['class_name'],
                    'coordinates': layout['coordinates'],
                    'text': text
                })

        result_text = f"✅ OCR 분석 완료!\n🔤 언어: {language}\n📄 추출된 텍스트 블록: {len(global_ocr_results)}개\n\n"

        # 미리보기 (처음 3개)
        for i, result in enumerate(global_ocr_results[:3]):
            result_text += f"📝 블록 {i+1} ({result['class_name']}):\n"
            preview = result['text'][:100].replace('\n', ' ')
            if len(result['text']) > 100:
                preview += "..."
            result_text += f"   {preview}\n\n"

        if len(global_ocr_results) > 3:
            result_text += f"... 및 {len(global_ocr_results)-3}개 더"

        logger.success(f"✅ OCR 완료: {len(global_ocr_results)}개 블록")

        return result_text

    except Exception as e:
        logger.error(f"❌ OCR 분석 실패: {e}")
        return f"❌ OCR 분석 실패: {e}"

def run_ai_api_analysis(api_key):
    """AI API 분석 실행"""
    global global_layout_info, global_img, global_api_results

    try:
        if not api_key:
            return "❌ OpenAI API 키를 입력해주세요!"

        if global_layout_info is None:
            return "❌ 먼저 레이아웃 분석을 실행해주세요!"

        logger.info("🤖 AI API 분석 시작...")

        import openai
        client = openai.OpenAI(api_key=api_key)

        # API 대상 클래스
        target_classes = ['figure', 'table']
        global_api_results = []

        # 클래스별 프롬프트
        prompts = {
            'figure': "이 그림(figure)의 내용을 간단히 요약해 주세요.",
            'table': "이 표(table)의 주요 내용을 요약해 주세요."
        }

        for layout in global_layout_info:
            if layout['class_name'].lower() not in target_classes:
                continue

            x1, y1, x2, y2 = layout['box']
            cropped_img = global_img[y1:y2, x1:x2]

            if cropped_img.size == 0:
                continue

            # 이미지를 base64로 인코딩
            pil_img = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
            buffered = io.BytesIO()
            pil_img.save(buffered, format="PNG")
            img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

            # 프롬프트 선택
            prompt = prompts.get(layout['class_name'].lower(), f"이 {layout['class_name']}의 내용을 간단히 설명해 주세요.")

            # API 호출
            response = client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[{
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_base64}"}}
                    ]
                }],
                max_tokens=300
            )

            description = response.choices[0].message.content.strip()

            global_api_results.append({
                'id': layout['id'],
                'class_name': layout['class_name'],
                'coordinates': layout['coordinates'],
                'description': description
            })

        result_text = f"✅ AI API 분석 완료!\n🤖 분석된 {len(global_api_results)}개 객체\n\n"

        for result in global_api_results:
            result_text += f"🖼️ {result['class_name']} (ID: {result['id']}):\n"
            preview = result['description'][:150].replace('\n', ' ')
            if len(result['description']) > 150:
                preview += "..."
            result_text += f"   {preview}\n\n"

        logger.success(f"✅ AI API 완료: {len(global_api_results)}개 객체")

        return result_text

    except Exception as e:
        logger.error(f"❌ AI API 분석 실패: {e}")
        return f"❌ AI API 분석 실패: {e}"

def integrate_all_results():
    """모든 결과 통합"""
    global global_ocr_results, global_api_results, global_img

    try:
        total_objects = len(global_ocr_results) + len(global_api_results)

        if total_objects == 0:
            return None, "❌ 분석할 결과가 없습니다. OCR 또는 API 분석을 먼저 실행해주세요!"

        logger.info("🔗 결과 통합 시작...")

        # 통합 문서 생성
        if global_img is not None:
            canvas = np.ones_like(global_img) * 255  # 흰색 캔버스

            # OCR 결과를 캔버스에 표시
            for result in global_ocr_results:
                x1, y1, x2, y2 = result['coordinates']
                cv2.rectangle(canvas, (x1, y1), (x2, y2), (0, 0, 0), 2)

                # 텍스트 간단히 표시 (최대 3줄)
                lines = result['text'].split('\n')[:3]
                for i, line in enumerate(lines):
                    if len(line) > 30:
                        line = line[:30] + "..."
                    cv2.putText(canvas, line, (x1+5, y1+20+i*20),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

            # API 결과를 캔버스에 표시
            for result in global_api_results:
                x1, y1, x2, y2 = result['coordinates']
                cv2.rectangle(canvas, (x1, y1), (x2, y2), (255, 0, 0), 2)

                # 설명 간단히 표시 (최대 2문장)
                desc_lines = result['description'].split('.')[:2]
                for i, line in enumerate(desc_lines):
                    if len(line) > 35:
                        line = line[:35] + "..."
                    cv2.putText(canvas, line, (x1+5, y1+20+i*20),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

            result_img = Image.fromarray(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
        else:
            result_img = None

        # 상세 통계 텍스트
        stats_text = f"🎉 결과 통합 완료!\n\n📊 전체 분석 객체: {total_objects}개\n"
        stats_text += f"   • OCR 텍스트 블록: {len(global_ocr_results)}개\n"
        stats_text += f"   • AI 분석 객체: {len(global_api_results)}개\n\n"

        if global_ocr_results:
            stats_text += "📋 OCR 결과 요약:\n"
            for result in global_ocr_results:
                preview = result['text'][:50].replace('\n', ' ')
                if len(result['text']) > 50:
                    preview += "..."
                stats_text += f"   • {result['class_name']}: {preview}\n"

        if global_api_results:
            stats_text += "\n🤖 AI 분석 결과 요약:\n"
            for result in global_api_results:
                preview = result['description'][:50].replace('\n', ' ')
                if len(result['description']) > 50:
                    preview += "..."
                stats_text += f"   • {result['class_name']}: {preview}\n"

        stats_text += "\n💾 결과가 메모리에 저장되었습니다."

        logger.success(f"✅ 결과 통합 완료: {total_objects}개 객체")

        return result_img, stats_text

    except Exception as e:
        logger.error(f"❌ 결과 통합 실패: {e}")
        return None, f"❌ 결과 통합 실패: {e}"

def reset_all_data():
    """전체 데이터 초기화"""
    global global_model, global_device, global_image_path, global_img
    global global_layout_info, global_ocr_results, global_api_results

    # 메모리 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    global_model = None
    global_device = None
    global_image_path = None
    global_img = None
    global_layout_info = None
    global_ocr_results = []
    global_api_results = []

    logger.info("🔄 시스템 초기화 완료")

    return "🔄 시스템이 초기화되었습니다. 다시 환경 설정부터 시작하세요."

logger.info("✅ SmartEye 핵심 함수들 정의 완료!")

2025-06-04 04:58:04.867 | INFO     | __main__:<cell line: 0>:454 - ✅ SmartEye 핵심 함수들 정의 완료!


In [ ]:
# ============================================
# 🎨 Gradio 웹 UI 구성
# ============================================

def create_smarteye_interface():
    """SmartEye Gradio 인터페이스 생성"""

    with gr.Blocks(
        theme=gr.themes.Soft(),
        title="SmartEye 학습지 분석",
        css="""
        .gradio-container {
            max-width: 1200px !important;
        }
        .tab-nav {
            background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
        }
        """
    ) as demo:

        # 메인 헤더
        gr.Markdown("""
        # 🎯 SmartEye 학습지 분석 시스템
        ### DocLayout-YOLO + OpenAI Vision API 통합 데모 (Colab 버전)

        **📋 사용 순서**: ⚙️환경설정 → 🤖모델다운로드 → 📸이미지업로드 → 🔍레이아웃분석 → 📝OCR/🤖API분석 → 🔗결과통합
        """)

        # 전체 상태 표시
        with gr.Row():
            status_display = gr.Textbox(
                label="📊 시스템 상태",
                value="⏳ 시스템 준비 중... 먼저 '환경 설정'을 실행하세요.",
                lines=2,
                interactive=False,
                container=True
            )

        with gr.Tabs() as tabs:

            # 🚀 빠른 시작 탭
            with gr.Tab("🚀 빠른 시작"):
                gr.Markdown("""
                ### 🎯 SmartEye 사용법

                1. **⚙️ 환경 설정**: GPU 확인 및 시스템 준비
                2. **🤖 모델 설정**: 분석 모델 다운로드 (3-5분 소요)
                3. **🔍 이미지 분석**: 학습지 업로드 후 레이아웃 분석
                4. **📝 OCR 분석**: 텍스트 영역에서 문자 추출
                5. **🤖 AI 분석**: 그림/표를 AI로 설명 생성 (API 키 필요)
                6. **🔗 결과 통합**: 모든 분석 결과를 하나의 문서로 통합

                ### ⚠️ 주의사항
                - **GPU 권장**: 분석 속도 향상을 위해 GPU 런타임 사용 권장
                - **API 키**: OpenAI API 키가 있어야 그림/표 분석 가능
                - **메모리**: 대용량 이미지는 메모리 부족을 일으킬 수 있음
                - **세션**: Colab 세션 종료 시 모든 데이터 초기화됨
                """
                )

                with gr.Row():
                    quick_setup_btn = gr.Button("🔧 환경 설정", variant="primary", scale=1)
                    quick_reset_btn = gr.Button("🔄 전체 초기화", variant="stop", scale=1)

                quick_output = gr.Textbox(label="실행 결과", lines=5)

                quick_setup_btn.click(fn=setup_environment, outputs=quick_output)
                quick_reset_btn.click(fn=reset_all_data, outputs=quick_output)

            # ⚙️ 환경 설정 탭
            with gr.Tab("⚙️ 환경 설정"):
                gr.Markdown("### 🔧 시스템 환경 설정")

                with gr.Row():
                    setup_btn = gr.Button("🔧 환경 설정 실행", variant="primary", size="lg")

                setup_output = gr.Textbox(label="설정 결과", lines=8)

                gr.Markdown("""
                **환경 설정 내용:**
                - GPU/CPU 디바이스 확인
                - 메모리 상태 점검
                - Tesseract OCR 설정 확인
                - 로그 시스템 초기화
                """)

                setup_btn.click(fn=setup_environment, outputs=setup_output)

            # 🤖 모델 설정 탭
            with gr.Tab("🤖 모델 설정"):
                gr.Markdown("### 📥 DocLayout-YOLO 모델 다운로드")

                with gr.Row():
                    with gr.Column(scale=2):
                        model_choice = gr.Dropdown(
                            choices=[
                                "DocStructBench (학습지 최적화)",
                                "DocLayNet-Docsynth300K (일반문서)",
                                "DocSynth300K-pretrain (커스텀)"
                            ],
                            value="DocStructBench (학습지 최적화)",
                            label="🎯 모델 선택",
                            info="학습지 분석에는 DocStructBench 권장"
                        )

                    with gr.Column(scale=1):
                        download_btn = gr.Button("⬇️ 모델 다운로드", variant="primary", size="lg")

                model_output = gr.Textbox(label="다운로드 결과", lines=8)

                gr.Markdown("""
                **모델 설명:**
                - **DocStructBench**: 학습지와 교과서에 특화된 모델 (권장)
                - **DocLayNet-Docsynth300K**: 일반적인 문서 분석용 모델
                - **DocSynth300K-pretrain**: 커스터마이징을 위한 사전훈련 모델

                ⏰ **다운로드 시간**: 약 3-5분 (모델 크기: ~50MB)
                """)

                download_btn.click(
                    fn=download_model_real,
                    inputs=model_choice,
                    outputs=model_output
                )

            # 🔍 이미지 분석 탭
            with gr.Tab("🔍 이미지 분석"):
                gr.Markdown("### 📸 학습지 이미지 업로드 및 레이아웃 분석")

                with gr.Row():
                    with gr.Column(scale=1):
                        input_image = gr.Image(
                            label="📸 학습지 이미지 업로드",
                            type="pil",
                            height=400
                        )

                        confidence_slider = gr.Slider(
                            minimum=0.1,
                            maximum=0.9,
                            value=0.25,
                            step=0.05,
                            label="🎯 신뢰도 임계값",
                            info="낮을수록 더 많은 객체 감지 (노이즈 증가)"
                        )

                        analyze_btn = gr.Button("🔍 레이아웃 분석 시작", variant="success", size="lg")

                    with gr.Column(scale=1):
                        output_image = gr.Image(
                            label="📊 분석 결과 시각화",
                            height=400
                        )

                        analysis_output = gr.Textbox(
                            label="🔍 분석 결과",
                            lines=12
                        )

                # 이미지 업로드 시 자동 처리
                input_image.change(
                    fn=process_uploaded_image,
                    inputs=input_image,
                    outputs=[output_image, analysis_output]
                )

                # 레이아웃 분석 실행
                analyze_btn.click(
                    fn=analyze_layout_real,
                    inputs=confidence_slider,
                    outputs=[output_image, analysis_output]
                )

            # 📝 OCR & AI 분석 탭
            with gr.Tab("📝 OCR & AI 분석"):
                gr.Markdown("### 📝 텍스트 추출 및 🤖 AI 이미지 분석")

                with gr.Row():
                    # OCR 분석 섹션
                    with gr.Column(scale=1):
                        gr.Markdown("#### 📝 OCR 텍스트 추출")

                        ocr_language = gr.Dropdown(
                            choices=["kor+eng", "eng", "kor"],
                            value="kor+eng",
                            label="🔤 OCR 언어 설정",
                            info="한국어+영어 혼합 권장"
                        )

                        ocr_btn = gr.Button("📝 OCR 분석 시작", variant="info", size="lg")

                        ocr_output = gr.Textbox(
                            label="📄 OCR 결과",
                            lines=12,
                            placeholder="OCR 분석 결과가 여기에 표시됩니다..."
                        )

                    # AI API 분석 섹션
                    with gr.Column(scale=1):
                        gr.Markdown("#### 🤖 AI 이미지 분석")

                        api_key_input = gr.Textbox(
                            label="🔑 OpenAI API Key",
                            type="password",
                            placeholder="sk-...",
                            info="그림/표 분석을 위해 필요"
                        )

                        api_btn = gr.Button("🤖 AI 분석 시작", variant="secondary", size="lg")

                        api_output = gr.Textbox(
                            label="🖼️ AI 분석 결과",
                            lines=12,
                            placeholder="AI 분석 결과가 여기에 표시됩니다..."
                        )

                gr.Markdown("""
                **분석 대상:**
                - **OCR**: title, plain text, abandon text, table caption, table footnote 등
                - **AI API**: figure, table 등의 이미지 요소
                """)

                # 이벤트 연결
                ocr_btn.click(
                    fn=run_ocr_analysis,
                    inputs=ocr_language,
                    outputs=ocr_output
                )

                api_btn.click(
                    fn=run_ai_api_analysis,
                    inputs=api_key_input,
                    outputs=api_output
                )

            # 🔗 결과 통합 탭
            with gr.Tab("🔗 결과 통합"):
                gr.Markdown("### 📄 최종 결과 통합 및 문서화")

                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("""
                        #### 🔗 통합 옵션

                        **통합 내용:**
                        - OCR로 추출된 모든 텍스트
                        - AI가 분석한 그림/표 설명
                        - 원본 위치 정보 유지
                        - 통계 및 요약 정보
                        """)

                        integrate_btn = gr.Button(
                            "🔗 결과 통합 실행",
                            variant="primary",
                            size="lg"
                        )

                        gr.Markdown("""
                        #### 🔄 시스템 관리
                        """)

                        reset_btn = gr.Button(
                            "🔄 전체 초기화",
                            variant="stop",
                            size="lg"
                        )

                    with gr.Column(scale=2):
                        final_image = gr.Image(
                            label="📄 통합 결과 문서",
                            height=400
                        )

                        final_output = gr.Textbox(
                            label="📊 최종 분석 보고서",
                            lines=15,
                            placeholder="통합 결과가 여기에 표시됩니다..."
                        )

                # 이벤트 연결
                integrate_btn.click(
                    fn=integrate_all_results,
                    outputs=[final_image, final_output]
                )

                reset_btn.click(
                    fn=reset_all_data,
                    outputs=status_display
                )

        # 푸터
        gr.Markdown("""
        ---
        **🎯 SmartEye v1.0** | Powered by DocLayout-YOLO & OpenAI GPT-4V | Running on Google Colab

        💡 **팁**: 문제 발생 시 '전체 초기화' 후 다시 시도해보세요.
        """)

    return demo

logger.info("✅ Gradio UI 구성 완료!")

2025-06-04 04:58:04.887 | INFO     | __main__:<cell line: 0>:301 - ✅ Gradio UI 구성 완료!


In [ ]:
# ============================================
# 🚀 SmartEye Gradio UI 실행
# ============================================

# UI 인스턴스 생성
demo = create_smarteye_interface()

# Colab에서 공개 링크와 함께 실행
try:
    demo.launch(
        share=True,          # 공개 링크 생성 (Colab 필수)
        debug=True,          # 디버그 모드
        show_error=True,     # 에러 표시
        server_name="0.0.0.0",  # Colab 호스팅
        server_port=7860,    # 기본 포트
        inbrowser=True,      # 자동으로 브라우저 열기
        quiet=False          # 실행 로그 표시
    )

    print("\n" + "="*80)
    print("🎉 SmartEye Gradio UI가 성공적으로 실행되었습니다!")
    print("🔗 위의 'public URL'을 클릭하여 웹 UI에 접속하세요")
    print("📱 모바일/태블릿에서도 접속 가능합니다")
    print("💡 링크를 다른 사람과 공유할 수도 있습니다")
    print("⚠️  Colab 세션이 종료되면 링크도 만료됩니다")
    print("="*80)

except Exception as e:
    logger.error(f"❌ UI 실행 실패: {e}")
    print(f"\n❌ Gradio UI 실행 중 오류가 발생했습니다: {e}")
    print("\n🔧 해결 방법:")
    print("1. 런타임을 다시 시작하고 모든 셀을 순서대로 실행하세요")
    print("2. 포트가 충돌하는 경우 다른 포트를 사용해보세요")
    print("3. 메모리 부족 시 GPU 런타임을 사용하세요")

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://103b503b109e9f1261.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


2025-06-04 05:00:02.598 | INFO     | __main__:download_model_real:42 - ⬇️ 모델 다운로드 시작: doclayout_yolo_docstructbench_imgsz1024.pt
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)clayout_yolo_docstructbench_imgsz1024.pt:   0%|          | 0.00/40.7M [00:00<?, ?B/s]

2025-06-04 05:00:04.506 | SUCCESS  | __main__:download_model_real:55 - ✅ 모델 로드 완료: /root/.cache/huggingface/hub/models--juliozhao--DocLayout-YOLO-DocStructBench/snapshots/8c3299a30b8ff29a1503c4431b035b93220f7b11/doclayout_yolo_docstructbench_imgsz1024.pt
2025-06-04 05:00:18.748 | INFO     | __main__:process_uploaded_image:82 - ✅ 이미지 처리 완료: 4118x5914, 1925.8KB
2025-06-04 05:01:09.899 | INFO     | __main__:analyze_layout_real:147 - 🔍 레이아웃 분석 시작...



image 1/1 /tmp/uploaded_image.jpg: 1024x736 2 titles, 13 plain texts, 3 abandons, 9 figures, 134.5ms
Speed: 17.1ms preprocess, 134.5ms inference, 326.3ms postprocess per image at shape (1, 3, 1024, 736)


2025-06-04 05:01:11.816 | SUCCESS  | __main__:analyze_layout_real:200 - ✅ 레이아웃 분석 완료: 27개 영역
2025-06-04 05:01:19.625 | INFO     | __main__:run_ocr_analysis:216 - 📝 OCR 분석 시작: kor
2025-06-04 05:01:24.829 | SUCCESS  | __main__:run_ocr_analysis:264 - ✅ OCR 완료: 12개 블록
2025-06-04 05:01:26.753 | INFO     | __main__:run_ai_api_analysis:283 - 🤖 AI API 분석 시작...
2025-06-04 05:02:18.410 | SUCCESS  | __main__:run_ai_api_analysis:348 - ✅ AI API 완료: 9개 객체
2025-06-04 05:03:27.367 | INFO     | __main__:integrate_all_results:366 - 🔗 결과 통합 시작...
2025-06-04 05:03:27.516 | SUCCESS  | __main__:integrate_all_results:425 - ✅ 결과 통합 완료: 21개 객체


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://103b503b109e9f1261.gradio.live

🎉 SmartEye Gradio UI가 성공적으로 실행되었습니다!
🔗 위의 'public URL'을 클릭하여 웹 UI에 접속하세요
📱 모바일/태블릿에서도 접속 가능합니다
💡 링크를 다른 사람과 공유할 수도 있습니다
⚠️  Colab 세션이 종료되면 링크도 만료됩니다


# 📖 사용 가이드

## 🚀 빠른 시작

1. **모든 셀 실행**: `런타임` → `모두 실행` 클릭
2. **UI 접속**: 마지막 셀 실행 후 생성되는 공개 링크 클릭
3. **단계별 진행**: 웹 UI에서 탭 순서대로 분석 진행

## 📋 상세 사용법

### 1단계: 환경 설정
- `⚙️ 환경 설정` 탭에서 시스템 준비
- GPU 사용 가능 여부 확인

### 2단계: 모델 다운로드
- `🤖 모델 설정` 탭에서 원하는 모델 선택
- **DocStructBench** 권장 (학습지 최적화)
- 다운로드 시간: 3-5분

### 3단계: 이미지 분석
- `🔍 이미지 분석` 탭에서 학습지 이미지 업로드
- 신뢰도 임계값 조정 (기본값: 0.25)
- 레이아웃 분석 실행

### 4단계: 콘텐츠 분석
- `📝 OCR & AI 분석` 탭에서 텍스트 추출
- OpenAI API 키 입력 시 그림/표 AI 분석 가능

### 5단계: 결과 통합
- `🔗 결과 통합` 탭에서 최종 문서 생성
- 모든 분석 결과를 하나로 통합

## ⚠️ 주의사항

- **메모리 관리**: 대용량 이미지는 메모리 부족 유발 가능
- **API 제한**: OpenAI API 사용량 확인 필요
- **세션 유지**: Colab 세션 종료 시 모든 데이터 초기화
- **오류 발생**: '전체 초기화' 후 재시도

## 🔧 문제 해결

### 메모리 부족
```python
# GPU 메모리 정리
import torch
torch.cuda.empty_cache()
```

### 모델 다운로드 실패
- 인터넷 연결 상태 확인
- 런타임 재시작 후 재시도

### UI 접속 불가
- 공개 링크가 생성되었는지 확인
- 브라우저 캐시 삭제 후 재접속

---

**📞 지원**: 문제 발생 시 GitHub Issues 또는 이메일로 문의하세요.